Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

Chargement des données

In [2]:
# Définition des transformations à appliquer aux images
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertit les images en tenseurs PyTorch de dimensions [channels, height, width]
    transforms.Normalize((0.1307,), (0.3081,))  # Normalisation avec la moyenne et l'écart-type de MNIST
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

vérifier la dispersion dans chaque classe
rajouter un split test pour valider la recherche d'hyperparamètre pour valider les hyperparamètres : tailles des NN, dropout
utiliser optuna
augmentation des données
80/10/10 avec l'équilibre entre les classes

In [3]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

dataiter = iter(train_loader)
images, labels = next(dataiter)
print(f'Taille des images : {images.shape}')  # Doit être [64, 1, 28, 28]
print(f'Taille des labels : {labels.shape}')   # Doit être [64]

print(f'Images min/max : {images.min()}/{images.max()}')  # Doit être entre 0.0 et 1.0
print(f'Labels uniques : {labels.unique()}')

Taille des images : torch.Size([64, 1, 28, 28])
Taille des labels : torch.Size([64])
Images min/max : -0.4242129623889923/2.821486711502075
Labels uniques : tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


Création du modèle

In [4]:
class FlexibleCNN(nn.Module):
    def __init__(self, conv_layers_params, dense_layers_sizes, dropout_conv=0.0, dropout_dense=0.0):
        """
        CNN flexible avec ajout de dropout, où les paramètres des couches convolutionnelles et des couches denses peuvent être spécifiés.

        :param conv_layers_params: Liste de dictionnaires contenant les paramètres pour chaque couche convolutionnelle.
                                   Chaque dictionnaire doit avoir les clés 'out_channels', 'kernel_size', 'stride', et 'padding'.
        :param dense_layers_sizes: Liste contenant les tailles des couches denses après les convolutions.
                                   Par exemple, [128, 64] signifie deux couches denses avec 128 et 64 neurones respectivement.
        :param dropout_conv: Probabilité de dropout après les couches convolutionnelles (par défaut 0.0, pas de dropout).
        :param dropout_dense: Probabilité de dropout après les couches denses (par défaut 0.0, pas de dropout).
        """
        super(FlexibleCNN, self).__init__()
        self.conv_layers = nn.Sequential()
        in_channels = 1  # Pour MNIST, les images ont un canal en entrée
        for idx, conv_params in enumerate(conv_layers_params):
            # Création dynamique des couches convolutionnelles
            self.conv_layers.add_module(
                f'conv_{idx}',
                nn.Conv2d(
                    in_channels,
                    conv_params['out_channels'],
                    kernel_size=conv_params.get('kernel_size', 3),
                    stride=conv_params.get('stride', 1),
                    padding=conv_params.get('padding', 1)
                )
            )
            self.conv_layers.add_module(f'relu_{idx}', nn.ReLU())
            self.conv_layers.add_module(f'pool_{idx}', nn.MaxPool2d(kernel_size=2, stride=2))
            if dropout_conv > 0.0:
                self.conv_layers.add_module(f'dropout_conv_{idx}', nn.Dropout2d(p=dropout_conv))
            in_channels = conv_params['out_channels']

        # Calcul de la taille après les couches convolutionnelles pour aplatir correctement
        self.flatten_size = self._get_flatten_size()

        # Création dynamique des couches denses
        self.dense_layers = nn.Sequential()
        input_size = self.flatten_size
        for idx, size in enumerate(dense_layers_sizes):
            self.dense_layers.add_module(f'fc_{idx}', nn.Linear(input_size, size))
            self.dense_layers.add_module(f'relu_fc_{idx}', nn.ReLU())
            if dropout_dense > 0.0:
                self.dense_layers.add_module(f'dropout_dense_{idx}', nn.Dropout(p=dropout_dense))
            input_size = size
        # Dernière couche pour la classification
        self.dense_layers.add_module('output', nn.Linear(input_size, 10))  # 10 classes pour MNIST

    def _get_flatten_size(self):
        # Méthode pour calculer la taille du tenseur après les convolutions
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, 28, 28)  # Taille d'une image MNIST
            x = self.conv_layers(dummy_input)
            flatten_size = x.view(1, -1).size(1)
        return flatten_size

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, self.flatten_size)  # Aplatir pour les couches denses
        x = self.dense_layers(x)
        return x

Paramètres d'entraînement

In [5]:
# Exemple de paramètres pour un CNN flexible
conv_layers_params = [
    {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
    {'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 1},
    # Vous pouvez ajouter autant de couches que vous le souhaitez
]
dense_layers_sizes = [128, 64]  # Tailles des couches denses

# Instanciation du modèle flexible
model = FlexibleCNN(conv_layers_params, dense_layers_sizes)

# Vérification du modèle
print(model)

# Instanciation de la fonction de perte et de l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Test du modèle avec un batch d'images
output = model(images)
print(f'Taille de la sortie du modèle : {output.shape}')  # Doit être [64, 10]

# Boucle d'entraînement
num_epochs = 100
patience = 5
best_val_loss = float('inf')
patience_counter = 0

FlexibleCNN(
  (conv_layers): Sequential(
    (conv_0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_0): ReLU()
    (pool_0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_1): ReLU()
    (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense_layers): Sequential(
    (fc_0): Linear(in_features=3136, out_features=128, bias=True)
    (relu_fc_0): ReLU()
    (fc_1): Linear(in_features=128, out_features=64, bias=True)
    (relu_fc_1): ReLU()
    (output): Linear(in_features=64, out_features=10, bias=True)
  )
)
Taille de la sortie du modèle : torch.Size([64, 10])


c:\Users\emac\Documents\M2\DeepLearning\dl_venv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training loop

In [6]:
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    total_train = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)  # images de taille [batch_size, 1, 28, 28]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

        # Calcul de l'accuracy sur l'ensemble d'entraînement
        _, predicted = torch.max(outputs, 1)  # Prend l'indice avec la plus grande probabilité
        train_correct += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct / total_train
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0
    val_correct = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            # Calcul de l'accuracy sur l'ensemble de validation
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = 100.0 * val_correct / total_val
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model_mnist.pt')
        print(f'Meilleur modèle sauvegardé avec une perte de validation de {best_val_loss:.4f}')
        patience_counter = 0
    else:
        patience_counter += 1
        print(f'Patience counter: {patience_counter}. Meilleure perte de validation: {best_val_loss:.4f}')
        if patience_counter >= patience:
            print('Early stopping triggered. Fin de l\'entraînement.')
            break

Epoch 1/100, Train Loss: 0.1626, Train Accuracy: 94.95%, Validation Loss: 0.0527, Validation Accuracy: 98.34%
Meilleur modèle sauvegardé avec une perte de validation de 0.0527
Epoch 2/100, Train Loss: 0.0477, Train Accuracy: 98.51%, Validation Loss: 0.0453, Validation Accuracy: 98.54%
Meilleur modèle sauvegardé avec une perte de validation de 0.0453


Visualisation des résultats

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Courbe d\'entraînement et de validation')
plt.legend()
plt.show()

# Étape 12 : Charger le meilleur modèle et évaluer les performances
model.load_state_dict(torch.load('best_model_mnist.pt'))
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Précision sur le jeu de validation : {100 * correct / total:.2f}%')